# American Option Pricing and Greek Calculation Using Binomial Tree Model

## Overview: 
### An American option is a type of options contract that can be exercised at any time before expiration. This flexibility requires a more sophisticated pricing model compared to European options. The binomial tree model is a popular method for this purpose, which discretizes the option's life into several time steps and models the underlying asset price movements.

In [1]:
import numpy as np

In [3]:
def binomial_tree_american_option(S, K, T, r, sigma, N, option_type='call'):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = np.exp(-sigma * np.sqrt(dt))
    p = (np.exp(r * dt) - d) / (u - d)
     # Initialize asset prices at maturity
    asset_prices = np.zeros(N + 1)
    option_values = np.zeros(N + 1)
    
    #Calculates asset prices at maturity for each possible path.
    for j in range(N + 1):
        asset_prices[j] = S * (u ** j) * (d ** (N - j))
    
    # Initialize option values at maturity
    if option_type == 'call':
        for j in range(N + 1):
            option_values[j] = max(0, asset_prices[j] - K)
    elif option_type == 'put':
        for j in range(N + 1):
            option_values[j] = max(0, K - asset_prices[j])
    
    # Backward Iteration through the tree
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            option_values[j] = np.exp(-r * dt) * (p * option_values[j + 1] + (1 - p) * option_values[j])
            asset_prices[j] = asset_prices[j] / u
            if option_type == 'call':
                option_values[j] = max(option_values[j], asset_prices[j] - K)
            elif option_type == 'put':
                option_values[j] = max(option_values[j], K - asset_prices[j])
    
    return option_values[0]

1. **Initialization:**
   - **Parameters**:
     - `dt = T / N`: Time step size.
     - `u = np.exp(sigma * np.sqrt(dt))`: Up factor.
     - `d = np.exp(-sigma * np.sqrt(dt))`: Down factor.
     - `p = (np.exp(r * dt) - d) / (u - d)`: Risk-neutral probability.

2. **Asset Prices at Maturity:**
   - **Calculation**: Computes asset prices at maturity for each path:
     - `asset_prices[j] = S * (u ** j) * (d ** (N - j))`

3. **Option Values at Maturity:**
   - **Initialization**: Depending on `option_type` ('call' or 'put'), initializes option values (`option_values[j]`) at maturity (`N`).

4. **Backward Iteration through the Tree:**
   - **Update Option Values**: Iterates backward through the tree, updating option values:
     - `option_values[j] = np.exp(-r * dt) * (p * option_values[j + 1] + (1 - p) * option_values[j])`
   - **Adjust Asset Prices**: Adjusts asset prices downward for each node in the current step.

5. **Early Exercise Check:**
   - Updates option values to consider early exercise opportunities based on `asset_prices[j]`.

6. **Return Value:**
   - Returns `option_values[0]`, the price of the American option at the initial node of the binomial tree.

In [4]:
# Example usage
S = 100  # Current stock price
K = 100  # Strike price
T = 1    # Time to maturity (1 year)
r = 0.05  # Risk-free interest rate
sigma = 0.2  # Volatility
N = 100  # Number of steps in the binomial tree


In [5]:
call_price = binomial_tree_american_option(S, K, T, r, sigma, N, option_type='call')
put_price = binomial_tree_american_option(S, K, T, r, sigma, N, option_type='put')

In [6]:

print(f"American Call Option Price: {call_price}")
print(f"American Put Option Price: {put_price}")

American Call Option Price: 10.430611662249142
American Put Option Price: 98.16843611112658


In [7]:
def calculate_greeks(S, K, T, r, sigma, N, option_type='call'):
    delta_S = 1e-4 * S  # Small change in S
    delta_t = 1e-4 * T  # Small change in T
    delta_sigma = 1e-4 * sigma  # Small change in sigma
    delta_r = 1e-4 * r  # Small change in r
    
    # Calculate option prices for perturbations
    price = binomial_tree_american_option(S, K, T, r, sigma, N, option_type)
    
    price_up_S = binomial_tree_american_option(S + delta_S, K, T, r, sigma, N, option_type)
    price_down_S = binomial_tree_american_option(S - delta_S, K, T, r, sigma, N, option_type)
    
    price_up_t = binomial_tree_american_option(S, K, T - delta_t, r, sigma, N, option_type)
    
    price_up_sigma = binomial_tree_american_option(S, K, T, r, sigma + delta_sigma, N, option_type)
    
    price_up_r = binomial_tree_american_option(S, K, T, r + delta_r, sigma, N, option_type)
    
    # Calculate Greeks
    delta = (price_up_S - price_down_S) / (2 * delta_S)
    gamma = (price_up_S - 2 * price + price_down_S) / (delta_S ** 2)
    theta = (price_up_t - price) / delta_t
    vega = (price_up_sigma - price) / delta_sigma
    rho = (price_up_r - price) / delta_r
    
    return {
        'Delta': delta,
        'Gamma': gamma,
        'Theta': theta,
        'Vega': vega,
        'Rho': rho
    }

1. **Initialization:**
   - **Parameters**:
     - `delta_S = 1e-4 * S`: Small change in the underlying asset price (`S`).
     - `delta_t = 1e-4 * T`: Small change in time to maturity (`T`).
     - `delta_sigma = 1e-4 * sigma`: Small change in volatility (`sigma`).
     - `delta_r = 1e-4 * r`: Small change in the risk-free interest rate (`r`).

2. **Calculate Option Prices for Perturbations:**
   - Computes the initial price of the American option:
     ```python
     price = binomial_tree_american_option(S, K, T, r, sigma, N, option_type)
     ```
   - Perturbs each parameter (`S`, `T`, `sigma`, `r`) and computes corresponding option prices:
     ```python
     price_up_S = binomial_tree_american_option(S + delta_S, K, T, r, sigma, N, option_type)
     price_down_S = binomial_tree_american_option(S - delta_S, K, T, r, sigma, N, option_type)
     price_up_t = binomial_tree_american_option(S, K, T - delta_t, r, sigma, N, option_type)
     price_up_sigma = binomial_tree_american_option(S, K, T, r, sigma + delta_sigma, N, option_type)
     price_up_r = binomial_tree_american_option(S, K, T, r + delta_r, sigma, N, option_type)
     ```

3. **Calculate Greeks:**
   - **Delta (`Δ`)**:
     ```python
     delta = (price_up_S - price_down_S) / (2 * delta_S)
     ```
   - **Gamma (`Γ`)**:
     ```python
     gamma = (price_up_S - 2 * price + price_down_S) / (delta_S ** 2)
     ```
   - **Theta (`Θ`)**:
     ```python
     theta = (price_up_t - price) / delta_t
     ```
   - **Vega (`ν`)**:
     ```python
     vega = (price_up_sigma - price) / delta_sigma
     ```
   - **Rho (`ρ`)**:
     ```python
     rho = (price_up_r - price) / delta_r
     ```

4. **Return Values:**
   - Returns a dictionary containing the calculated Greeks.

In [8]:
call_greeks = calculate_greeks(S, K, T, r, sigma, N, option_type='call')
put_greeks = calculate_greeks(S, K, T, r, sigma, N, option_type='put')

In [9]:
print(f"American Call Option Greeks: {call_greeks}")
print(f"American Put Option Greeks: {put_greeks}")

American Call Option Greeks: {'Delta': 0.6365119623631621, 'Gamma': 7.486003950827325, 'Theta': -6.404135864084282, 'Vega': 37.43011807104679, 'Rho': 53.22091994237609}
American Put Option Greeks: {'Delta': -0.018315638888566355, 'Gamma': 0.0, 'Theta': -3.6635857158273666, 'Vega': 36.62395249861561, 'Rho': 0.0}
